## ライブラリ導入

In [ ]:
from qiskit import *
from qiskit_aer import Aer
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from scipy.optimize import minimize

from commonLogic import *
from maxCut_common import *

%matplotlib inline

## パラメータ設定

In [ ]:
# ノード数
num = 5
# エッジ構造
edges = [(0,1),(0,2),(1,2),(1,3),(2,4),(3,4)]
# 重み
weights = [1,1,1,1,1,1]

# (γ,β)初期値
args_init = np.array([0.1,0.1,
                      0.1,0.1,
                      0.1,0.1])

## グラフ描画

In [ ]:
G = nx.Graph()
G.add_edges_from(edges)
pos = nx.spring_layout(G)
colors = ['aquamarine' for i in range(num)]
nx.draw(G, pos, with_labels=True, node_color=colors)

edge_labels = {}
for i in range(len(edges)):
    edge_labels[edges[i]] = weights[i]
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)

plt.show()

## 実装

In [ ]:
# コストハミルトニアン
operator_costHamiltonian = get_operator_costHamiltonian(num,edges,weights)

# 期待値の計算
def get_expectation_value(args):
    qc = get_mainCircuit(num,edges,weights,args)
    backend = Aer.get_backend("statevector_simulator")
    qc_transpiled = transpile(qc,backend)
    result = backend.run(qc_transpiled).result()
    statevector = result.get_statevector()
    expectation_value = statevector.expectation_value(operator_costHamiltonian).real
    return expectation_value

In [ ]:
# 古典最適化アルゴリズムで(γ,β)を最適化
optimizationResult = minimize(get_expectation_value, args_init, options={'maxiter':500}, method='powell')

print(f"{optimizationResult.fun:.2f}")
args_optimized = optimizationResult.x
print([float(i) for i in args_optimized])

In [ ]:
# 回路作成
qc = get_mainCircuit(num,edges,weights,args_optimized)
qc.measure(range(num),range(num))
qc.draw('mpl')

In [ ]:
# 最適値のシミュレーション実行
backend = Aer.get_backend("qasm_simulator")
qc_transpiled = transpile(qc,backend)
result = backend.run(qc_transpiled).result()

counts=result.get_counts()
for i in range(2**num):
    binary_string = format(i, f'0{num}b')
    if binary_string not in counts:
        counts[binary_string]=0
counts = dict(sorted(counts.items()))

my_plot_histogram(counts,gradation=True)

## 分割後のグラフ

In [ ]:
max_key = max(counts, key=counts.get)

node_colors = []
for i in max_key:
    if i=='0': node_colors.append('aquamarine')
    else: node_colors.append('orange')

edge_colors = []
for u, v in G.edges():
    if node_colors[u] != node_colors[v]: edge_colors.append('red')
    else: edge_colors.append('black')

nx.draw(G, pos, with_labels=True, node_color=node_colors, edge_color=edge_colors)

edge_labels = {}
for i in range(len(edges)):
    edge_labels[edges[i]] = weights[i]
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)

plt.show()